Cedric Keller***REMOVED*** & Alexander Zank***REMOVED***
# Assignment 5

This assignment is about visible light communication (VLC). LEDs act as transceivers and communicate using protocols that ensure no difference to "steady" light is visible to humans.

## Step 2

What is the bandwidth of the optical spectrum?

	The optical or visible spectrum is the range of wavelengths that are visible to the human eye.
	It ranges from 400THz (red) to around 800THz (violet). 
	In total we therefore get a bandwidth of around 400THZ, which is a lot!	
	
	We can estimate the maximum throughput to be around 400 terabits/s using Stefan Mangold's rule of thumb.
	
	
Is the visible spectrum regulated?

	Conserning frequency, the visible spectrum is not regulated.
	Powerful lightsources will however be regulated (such as strong lasers).	
	
	
What is the difference between infrared and visible light?

	Light in the frequency range of 300GHz to 400 THz is classified as infrared light.
	Infrared light has a lower frequency than visible light and is therefore not visible
	to humans. 
	
	
Can infrared light penetrate water? Can visible light?

	Looking at the graph: [wikipedia](https://en.wikipedia.org/wiki/Electromagnetic_absorption_by_water#/media/File:Absorption_spectrum_of_liquid_water.png)
	We can see that the visible spectrum is absorbed the most in water.
	This implies that visible light can barely penetrate water.
	50% of it is already absorbed within the first 10 meters. 
	Blue light is absorbed least and may reach depths of 100 meters.
	
	Infrared light is absorbed way less which means it is able to penetrate much deeper.
	
	
How do submarines communicate through the ocean, when they operate below water surface?

	There are different possibilities depending on the depth of the submarine.
	If it is near the water surface it may use (blue) light.
	
	Since light becomes less viable for lower depths,
	sonar (physical waves) might be used.
	
	There is also the possibility of extremely low frequency radio waves,
	but these require huge antennas which consume enormous amounts of power.
	
	
How can an LED be used as a receiver?

	A LED behaves like a capacitor; It will slowly discharge after it is charged.
	The LED discharges faster if there is incomming light. This effect can be measured 
	and can therefore LEDs can be used to detect incomming light.
	
	
What are the benefits of using an LED instead of a photodiode as a receiver in consumer electronics?

	In consumer electronics cost plays a big role. If we are able to reuse the LED as a receiver, 
	we do not have to add additional hardware in the form of a photodiode. 
	This massively reduces cost especially in products where it is for example very expensive to add a hole
	for a photodiode. 
	With the LED as a receiver we simply have to change the microchip without further costs.
	
	

